# IANNWTF - Homework 05
(by Group 17 - Nils Niehaus, Philipp Bauer, Marlon Dammann)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

## 1 Data Set

Constructing a Data Pipeline.
We load the dataset 'fashion_mnist' as supervised:

In [2]:
train_ds, test_ds = tfds.load('fashion_mnist', split=['train', 'test'], as_supervised=True)

and apply the appropriate preprocessing including normalizing, shuffling, prefetching and batching the data as well as onehotifying the targets:

In [3]:
def preprocess_fashion_mnist_dataset(data_split):
    '''We change the data type of the image values to float32'''
    data_split = data_split.map(lambda img, target: (tf.cast(img, 'float32'),target))
    '''We one-hotify the target and make the vector 2-dimensional'''
    data_split = data_split.map(lambda img, target: (img, tf.one_hot(target,10)))
    '''We normalize the gray-values to a range of 0-1'''
    data_split = data_split.map(lambda img, target: (tf.math.l2_normalize(img),target))    
    '''We take a batch size of 16 and prefetch 10 elements, shuffle is not necessary since it has been performed before'''
    data_split = data_split.shuffle(1000)
    data_split = data_split.batch(32)
    data_split = data_split.prefetch(20)
    return data_split

In [4]:
data_train = train_ds.apply(preprocess_fashion_mnist_dataset)
data_test = test_ds.apply(preprocess_fashion_mnist_dataset)

## 2 Model

Our CNN-Model is defined as a subclass from the standard keras model and consits of 2 Convolutional layers with given parameters (ref. to homework-pdf) followed by a Global-Average-Pooling layer. Those are surrounded by a dense input layer with a shape matching that of the image-data and a dense output layer.

In [5]:
class FashionMNISTClassifier(tf.keras.Model):
    '''Our wine-classifier model with adjustable size.'''
    def __init__(self):
        '''Initializes the model with a given down/upscaling of the standard size from the last task.'''
        
        super(FashionMNISTClassifier, self).__init__()
        
        self.input_layer = tf.keras.layers.Dense(units=28*28, activation=tf.nn.sigmoid)
        self.conv1 = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same')
        self.maxpool = tf.keras.layers.MaxPooling2D()
        self.conv2 = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same')
        self.maxpool2 = tf.keras.layers.MaxPooling2D()
        self.conv3 = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),strides=(1,1),padding='same')
        self.maxpool3 = tf.keras.layers.MaxPooling2D()
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.output_layer = tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)

    def call(self, inputs):
        '''The model\'s call method for forwarding the input through the layers.'''
        x = self.input_layer(inputs)
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.avgpool(x)
        x = self.output_layer(x)
        return x

## 3 Training

In [6]:
model_performance = {'training': {'loss' : [], 'accuracy' : []}, 
                     'test': {'loss' : [], 'accuracy' : []}}

In [7]:
dataset = {'training' : data_train, 'test' : data_test}

In [8]:
def get_loss_and_accuracy(model, data_split, loss_function, mode_name):
    '''Returns the loss and accuracy of the model on a given split of the data.'''
    accuracy_aggregator = []
    loss_aggregator = []

    for (input, target) in tqdm(data_split,desc='Sampling Loss/Accuracy for ' + str(mode_name) + ' data'):
        prediction = model(input)
        sample_loss = loss_function(target, prediction)
        sample_accuracy =  np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
        sample_accuracy = np.mean(sample_accuracy)
        loss_aggregator.append(sample_loss.numpy())
        accuracy_aggregator.append(np.mean(sample_accuracy))
        
    loss = tf.reduce_mean(loss_aggregator)
    accuracy = tf.reduce_mean(accuracy_aggregator)
    return loss, accuracy

In [9]:
def performance_test(model, data_split, loss_function, model_performance, mode_name):
    '''Evaluation of loss and accuracy of a model on a data split.'''
    loss, accuracy = get_loss_and_accuracy(model, data_split, loss_function, mode_name)
    model_performance[mode_name]['loss'].append(loss.numpy())
    model_performance[mode_name]['accuracy'].append(accuracy.numpy())
    if mode_name == 'test':
      print('Test accurady for current epoch: ', model_performance[mode_name]['accuracy'][-1])

In [10]:
def train_step(model, input, target, loss_function, optimizer):
    with tf.GradientTape() as tape:
        prediction = model(input)
        loss = loss_function(target, prediction)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [11]:
def train_model(model, dataset, optimizer, loss_function, num_epochs, model_performance):
    
    '''Initial performance of the model without training'''
    for data, name in zip(list(dataset.values()),list(dataset.keys())):
        performance_test(model, data, loss_function, model_performance, mode_name=name)
    
    for epoch in range(num_epochs):
        for input,target in tqdm(dataset['training'],desc='Epoch '+str(epoch+1)):
            train_step(model, input, target, loss_function, optimizer)
        for data, name in zip(list(dataset.values()),list(dataset.keys())):
            performance_test(model, data, loss_function, model_performance, mode_name=name)

In [12]:
tf.keras.backend.clear_session()

'''Hyperparameters'''
num_epochs = 10
learning_rate = 0.1

'''Loss function'''
cat_cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()

'''Adam as chosen optimizer'''
optimizer = tf.keras.optimizers.Adam(learning_rate)

'''Model'''
model = FashionMNISTClassifier()

In [13]:
train_model(model, dataset, optimizer, cat_cross_entropy_loss, num_epochs, model_performance)

Sampling Loss/Accuracy for test data: 100%|██████████| 313/313 [00:06<00:00, 45.51it/s]


Test accurady for current epoch:  0.10023961661341853


Sampling Loss/Accuracy for test data: 100%|██████████| 313/313 [00:05<00:00, 54.40it/s]


Test accurady for current epoch:  0.09984025559105432


Epoch 2:   6%|▌         | 112/1875 [00:04<01:11, 24.63it/s]


KeyboardInterrupt: ignored

## 4 Visualization

Plotting the accuarcy and loss results via matplotlib.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15, 4))
line1, = ax[0].plot(model_performance['training']['loss'])
line2, = ax[0].plot(model_performance['test']['loss'])
line3, = ax[1].plot(model_performance['training']['accuracy'])
line4, = ax[1].plot(model_performance['test']['accuracy'])
ax[0].set_xlabel("Training epoch")
ax[0].set_ylabel("Loss")
ax[1].set_xlabel("Training epoch")
ax[1].set_ylabel("Accuracy")
ax[0].legend((line1,line2),("training","test"))
ax[1].legend((line3,line4),("training","test"))
plt.show()

In [ ]:
print(model_performance['test']['accuracy'])

In [ ]:
model_performance['training']['loss']